# Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. На какое-то время жители городов перестали выходить на улицу, посещать кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Описание данных
Таблица books
Содержит данные о книгах:

    book_id — идентификатор книги;
    author_id — идентификатор автора;
    title — название книги;
    num_pages — количество страниц;
    publication_date — дата публикации книги;
    publisher_id — идентификатор издателя.

Таблица authors
Содержит данные об авторах:

    author_id — идентификатор автора;
    author — имя автора.

Таблица publishers
Содержит данные об издательствах:

    publisher_id — идентификатор издательства;
    publisher — название издательства;

Таблица ratings
Содержит данные о пользовательских оценках книг:

    rating_id — идентификатор оценки;
    book_id — идентификатор книги;
    username — имя пользователя, оставившего оценку;
    rating — оценка книги.

Таблица reviews
Содержит данные о пользовательских обзорах:

    review_id — идентификатор обзора;
    book_id — идентификатор книги;
    username — имя автора обзора;
    text — текст обзора.



Цель исследования: проанализировать имеющуюся базы данных электронных книг для дальнейших продаж пользователям.
    

Задачи:  
    1.Посчитать, сколько книг вышло после 1 января 2000 года;  
    2.Для каждой книги посчитать количество обзоров и среднюю оценку;  
    3.Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;  
    4.Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;  
    5.Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.  

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [2]:
query = '''SELECT * from 
 books'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


##  Посчитать, сколько книг вышло после 1 января 2000 года;  


In [3]:
query= '''SELECT count(book_id) FROM books  where publication_date> '2000-01-01' '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


## Для каждой книги посчитать количество обзоров и среднюю оценку; 

In [4]:
query= '''
SELECT books.book_id, count(distinct(review_id)), avg(rating) 
FROM books
left join reviews ON books.book_id=reviews.book_id 
left join ratings ON books.book_id=ratings.book_id
Group BY books.book_id'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,count,avg
0,1,2,3.666667
1,2,1,2.500000
2,3,3,4.666667
3,4,2,4.500000
4,5,4,4.000000
...,...,...,...
995,996,3,3.666667
996,997,3,3.400000
997,998,4,3.200000
998,999,2,4.500000


## Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;  


In [5]:
query= '''

WITH a as(SELECT publisher, count(book_id) as count_books
FROM books
join publishers ON books.publisher_id=publishers.publisher_id 
where num_pages>50
group by publisher)

select publisher, count_books from a
where count_books=(select max(count_books)
from a) 
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count_books
0,Penguin Books,42


## Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;  


In [6]:
query= '''
with a as(
select author,avg(rating)
from books
join authors ON books.author_id=authors.author_id
join ratings ON books.book_id=ratings.book_id
group by author
having count(rating) >50)

select author, avg 
from a
where avg=(select max(avg) from a)
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.288462


## Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.  

In [7]:
query = '''
WITH
    ratings_subquery AS (
        SELECT
            ratings.username,
            COUNT(rating) AS ratings_quantity
        FROM
            ratings
        GROUP BY
            ratings.username
        HAVING
            COUNT(rating) > 48
        ),
    reviews_subquery AS (
        SELECT
            reviews.username,
            COUNT(review_id) AS reviews_quantity
        FROM
            reviews
        GROUP BY
            reviews.username
    )
SELECT
    AVG(reviews_quantity)
FROM
    ratings_subquery
    LEFT JOIN reviews_subquery ON reviews_subquery.username = ratings_subquery.username
   '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con) 

,avg
0,24.0


Таким образом, проанализировав полученную базу данных книг, можем отметить, что после 1.01.2000 г. вышло 819 книг.  
Количество обзоров и средняя оценка для кажой книги представлены выше в таблице задания 2.  
Издательством, выпустившим наибольшее число книг толще 50 страниц, стало Penguin Books c количеством книг 42.  
Автором, с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками, стала J.K. Rowling/Mary GrandPré с оценкой 4.28.
Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок, составило 24.    